
# 11 — Multi-Agent Routing (ENT + Legal + Generic RAG)

This notebook demonstrates a simple multi-agent router that:
- Classifies a user question into ENT / Legal / Generic
- Routes the query to the corresponding handler
- Uses a LangGraph workflow as the orchestration layer

The ENT and Legal handlers here are lightweight stubs compatible with the project skeletons in Notebook 09.



## 1. Setup: Simple RAG Backends for Generic, ENT, and Legal

In a real project:
- ENT/Legal handlers would reuse the full pipelines from Notebook 09.
Here we create small stand-in handlers.


In [ ]:

!pip install langchain langchain-openai chromadb langchain-community langgraph --quiet

import os
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

os.makedirs("router_data", exist_ok=True)

# Generic corpus
with open("router_data/generic.txt", "w") as f:
    f.write("This is a generic RAG backend for general questions.\n")

# ENT corpus
with open("router_data/ent.txt", "w") as f:
    f.write("Chronic otitis media: management includes hearing assessment and infection control.\n")

# Legal corpus
with open("router_data/legal.txt", "w") as f:
    f.write("Limitation of liability: liability is capped at fees paid in the last 12 months.\n")

def build_db(path: str):
    loader = TextLoader(path)
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
    chunks = splitter.split_documents(docs)
    emb = OpenAIEmbeddings()
    return Chroma.from_documents(chunks, emb)

db_generic = build_db("router_data/generic.txt")
db_ent = build_db("router_data/ent.txt")
db_legal = build_db("router_data/legal.txt")

llm = ChatOpenAI(model="gpt-4o-mini")



## 2. Domain Classifier and Handlers

- Classifier: ENT vs Legal vs Generic
- Handlers: ENT, Legal, Generic answer functions


In [ ]:

def classify_domain(question: str) -> str:
    q = question.lower()
    if any(w in q for w in ["ear", "sinus", "throat", "otitis", "ent"]):
        return "ENT"
    if any(w in q for w in ["contract", "liability", "confidentiality", "governing law", "nda", "msa"]):
        return "LEGAL"
    return "GENERIC"

def answer_generic(question: str) -> str:
    docs = db_generic.similarity_search(question, k=2)
    ctx = "\n".join(d.page_content for d in docs)
    prompt = f"Context:\n{ctx}\n\nQuestion: {question}\nGeneral answer:"
    return llm.invoke(prompt).content

def answer_ent(question: str) -> str:
    docs = db_ent.similarity_search(question, k=3)
    ctx = "\n".join(d.page_content for d in docs)
    prompt = f"""You are an ENT guideline-style assistant.
Use ONLY the context below. Do not give diagnosis or personalized advice.

Context:
{ctx}

Question: {question}

Answer briefly in guideline style and clearly state this is not medical advice.
"""
    return llm.invoke(prompt).content

def answer_legal(question: str) -> str:
    docs = db_legal.similarity_search(question, k=3)
    ctx = "\n".join(d.page_content for d in docs)
    prompt = f"""You are a contract analysis assistant for legal professionals.
Use ONLY the context below.

Context:
{ctx}

Question: {question}

Summarize the relevant clause in neutral language and clearly state this is not legal advice.
"""
    return llm.invoke(prompt).content

print(classify_domain("How is chronic otitis media treated?"))
print(classify_domain("What does the limitation of liability clause mean?"))
print(classify_domain("What is RAG?"))



## 3. LangGraph Multi-Agent Router

We define a state and nodes:
- router_node: decides domain
- ent_node / legal_node / generic_node: call the proper handler


In [ ]:

from typing import TypedDict
from langgraph.graph import StateGraph, END

class RouterState(TypedDict):
    question: str
    domain: str
    answer: str

def router_node(state: RouterState) -> RouterState:
    state["domain"] = classify_domain(state["question"])
    return state

def ent_node(state: RouterState) -> RouterState:
    state["answer"] = answer_ent(state["question"])
    return state

def legal_node(state: RouterState) -> RouterState:
    state["answer"] = answer_legal(state["question"])
    return state

def generic_node(state: RouterState) -> RouterState:
    state["answer"] = answer_generic(state["question"])
    return state

graph = StateGraph(RouterState)
graph.add_node("router", router_node)
graph.add_node("ent_handler", ent_node)
graph.add_node("legal_handler", legal_node)
graph.add_node("generic_handler", generic_node)

graph.set_entry_point("router")

# Conditional routing
def route_from_router(state: RouterState):
    if state["domain"] == "ENT":
        return "ent_handler"
    if state["domain"] == "LEGAL":
        return "legal_handler"
    return "generic_handler"

graph.add_conditional_edges(
    "router",
    route_from_router,
    {
        "ent_handler": "ent_handler",
        "legal_handler": "legal_handler",
        "generic_handler": "generic_handler",
    },
)

graph.add_edge("ent_handler", END)
graph.add_edge("legal_handler", END)
graph.add_edge("generic_handler", END)

multi_agent_workflow = graph.compile()

# Test runs
print(multi_agent_workflow.invoke({"question": "How is chronic otitis media treated?"}))
print(multi_agent_workflow.invoke({"question": "What does the limitation of liability clause say?"}))
print(multi_agent_workflow.invoke({"question": "Explain retrieval-augmented generation."}))



## End of Notebook 11  
Multi-Agent routing across ENT / Legal / Generic RAG backends is ready.
